# Downloading and uploading a project
In this notebook, we will see how to download all available data for a project (media, annotations, predictions, models and the configuration) to our local machine. Once the project data is downloaded, we have effectively made a back-up of our project from the platform to the local disk. 

At the end of the notebook, we will use the downloaded data to re-create the project on the platform again, and upload all media and annotations to it. This could for example be used to migrate a project to a different server, or to make sure the project data is safe if a server outage is expected for whatever reason.

In [1]:
# As usual we will connect to the platform first, using the server details from the .env file. We will also create a ProjectClient for the server

from geti_sdk import Geti
from geti_sdk.rest_clients import ProjectClient
from geti_sdk.utils import get_server_details_from_env

geti_server_configuration = get_server_details_from_env()

geti = Geti(server_config=geti_server_configuration)

project_client = ProjectClient(session=geti.session, workspace_id=geti.workspace_id)

C:\Intel\GitHub\geti-sdk\geti_sdk\http_session\geti_session.py:71: InsecureRequestWarning: You have disabled TLS certificate validation, HTTPS requests made to the Intel® Geti™ server may be compromised. For optimal security, please enable certificate validation.
  warnings.warn(


2023-02-22 14:22:22,943 - INFO - Personal access token validated on host https://10.91.120.185


### Select project to download from the project list
As usual, let's list the available projects and pick one to download. You can change the selected project by changing the `PROJECT_NAME` variable, two cells down.

In [2]:
projects = project_client.list_projects()

8 projects were found on the platform:

 Project: bolt_detection
  Task 1: Detection task
    Labels: ['good', 'bad', 'No Object']


 Project: Bolts Anomaly Segmentation
  Task 1: Anomaly segmentation task
    Labels: ['Normal', 'Anomalous']


 Project: Weed Det-Class upload
  Task 1: Detection task
    Labels: ['pot', 'No Object']
  Task 2: Classification task
    Labels: ['broadleaf', 'grass', 'clover']


 Project: Counting coffee fruits
  Task 1: Instance segmentation task
    Labels: ['Immature', 'Semi-mature', 'Mature', 'Overmature', 'Empty']


 Project: Counting coffee fruits - trained
  Task 1: Instance segmentation task
    Labels: ['Immature', 'Semi-mature', 'Mature', 'Overmature', 'Empty']


 Project: Weeds Pot2Class
  Task 1: Detection task
    Labels: ['pot', 'No Object']
  Task 2: Classification task
    Labels: ['broadleaf', 'clover', 'grass']


 Project: Foam_detection
  Task 1: Detection task
    Labels: ['foam', 'No Object']


 Project: person-bike-car
  Task 1: Detect

Specify the name of the project to download. We will use the `COCO anomalous animal demo` project created in notebook [004](004_create_pipeline_project_from_dataset.ipynb).

In [3]:
#PROJECT_NAME = "AMR Detection in Warehouse - trained"
#PROJECT_NAME = "Weed Det-Class"
#PROJECT_NAME = "Counting coffee fruits"
#PROJECT_NAME = "Counting coffee fruits - trained"
PROJECT_NAME = "person-bike-car"

## Project download
Now, let's do the project download itself. The `Geti` provides a method `download_project()` to do so. It takes the following arguments:

- `project_name`: Name of the project to download
- `target_folder`: Path of the folder to download to. If left empty, a folder named `project_name` will be created in the current directory
- `include_predictions`: True to download predictions for all media, False to not download any predictions
- `include_active_models`: True to download the active models for all tasks in the project, False to not download any models
- `include_deployment`: True to create and download a deployment for the project, False to not create any deployment

Note that downloading may take some time especially for large projects, so please be patient. The method will print regular updates related to the download progress.

In [4]:
import os

project = geti.download_project(
    project_name=PROJECT_NAME,
    target_folder=os.path.join("projects", PROJECT_NAME),
    include_predictions=False,
    include_active_models=False,
    include_deployment=False,
)

2023-02-22 14:22:52,094 - INFO - Project parameters for project 'person-bike-car' were saved to file projects\person-bike-car\project.json.
2023-02-22 14:22:53,875 - INFO - Downloading 2 videos from project 'person-bike-car' to folder projects\person-bike-car\videos...


2023-02-22 14:23:06,140 - INFO - Downloaded 2 videos in 12.3 seconds.
2023-02-22 14:23:06,141 - INFO - Starting annotation download... saving annotations for 2 videos to folder projects\person-bike-car/annotations


2023-02-22 14:23:15,741 - INFO - Video annotation download finished in 8.8 seconds.
2023-02-22 14:23:18,206 - INFO - Project parameters for project 'person-bike-car' were saved to file projects\person-bike-car\configuration.json.
2023-02-22 14:23:18,207 - INFO - Project 'person-bike-car' was downloaded successfully.


That's all there is to it! Now, you should have a folder `projects` showing up in the current directory. Inside it should be another folder named `{PROJECT_NAME}`, which should contain all a file `project.json` holding the project details, as well as all media and annotations in the project and a file `configuration.json` that contains the full project configuration. 

In addition, the `download_project` method can also create a deployment for the project (see notebook [008 deploy_project](008_deploy_project.ipynb) for more details on deployments), if the parameter `include_deployment` is set to True. In that case you should see a folder called `deployment` in the project directory as well.

Note that in this case predictions, models and the deployment are not included because we have set `include_predictions=False`, `include_active_models=False` and `include_deployment=False`.

## Project upload
Now that the project is downloaded, we can use it to create a new project on the platform. Once the project is created, we can upload the media, annotations and configuration that were downloaded to the project folder to it. The `Geti` class provides a `upload_project()` method to do all that, which takes three parameters:
- `target_folder`: Path to the folder containing the project data to upload
- `project_name`: Optional name to assign to the new project on the platform. If left unspecified, the project name will correspond to the downloaded project name
- `enable_auto_train`: True to turn on auto training for all tasks in the project, once the media and annotation upload is complete. False to leave auto training turned off.

In [13]:
uploaded_project = geti.upload_project(
    target_folder=os.path.join("projects", PROJECT_NAME),
    project_name=PROJECT_NAME, 
    enable_auto_train=False,
)

2022-12-07 16:37:37,853 - INFO - Creating project 'Counting coffee fruits - trained' from parameters in configuration file at projects\Counting coffee fruits - trained\project.json.
2022-12-07 16:37:39,728 - INFO - Project created successfully.
2022-12-07 16:37:43,726 - INFO - Starting image upload...


Uploading images:   0%|          | 0/2 [00:00<?, ?it/s]

2022-12-07 16:37:46,360 - INFO - Upload complete. Uploaded 2 new images in 2.6 seconds.
2022-12-07 16:37:46,707 - INFO - Starting video upload...


Uploading videos:   0%|          | 0/2 [00:00<?, ?it/s]

2022-12-07 16:37:57,264 - INFO - Upload complete. Uploaded 2 new videos in 10.6 seconds.
2022-12-07 16:38:00,540 - INFO - Reading annotation files in folder projects\Counting coffee fruits - trained\annotations...
2022-12-07 16:38:00,658 - INFO - Starting image annotation upload...


Uploading image annotations:   0%|          | 0/2 [00:00<?, ?it/s]

2022-12-07 16:38:00,682 - INFO - No matching annotation file found for image with name scene00034_62fc0858406d791c511960b6. Skipping this image...
2022-12-07 16:38:02,375 - INFO - Upload complete. Uploaded 1 new image annotations
2022-12-07 16:38:02,378 - INFO - Waiting for the Geti server to process all uploaded videos...
2022-12-07 16:38:04,190 - INFO - Starting video annotation upload...


Uploading video frame annotations:   0%|          | 0/20 [00:00<?, ?it/s]

Uploading video frame annotations:   0%|          | 0/23 [00:00<?, ?it/s]

2022-12-07 16:39:09,634 - INFO - Upload complete. Uploaded 43 new video frame annotations
2022-12-07 16:39:11,317 - INFO - Project 'Counting coffee fruits - trained' was uploaded successfully.


Done! The uploaded project should now show up in your workspace. Note that it is of course also possible to upload the project to a different server, simply by setting up a new Geti instance to that server and calling the `upload_project()` method from that instance.